## Deteccion del tablero en la imagen
<pre>
Se usara el tablero B para esta seccion.
</pre>
### Objetivo:
<pre>
    * Se posee una armazon de metal que permite mantener una camara del microcontrolador ESP-32 CAM por encima de un tablero de 
    ajedrez. Este armazon consta de un circulo de metal como base. Lo que se desea es poder poner un tablero con una determinada
     posicion de piezas y que mediante una foto se pueda representar computacionalmente la situacion.
    
</pre>
-----------------------------------------
### Representaciones:
<pre>
Se usa una representacion de tablero mediante un array de 64 elementos de 8 bits cada uno. En cada uno de esos elementos se
 almacenara una pieza. Aqui se presentan los encodings para cada una de ellas. 
    * Los primeros 4 bits se usaran para representar el color.
        - 0000 XXXX : Negro
        - 1111 XXXX : Blanco
    * Con los 4 bits restantes se representa la pieza:
         Pieza        Codificacion
        - Peon      | 0000
        - Caballo   | 0001
        - Alfil     | 0010
        - Torre     | 0011
        - Reina     | 0100
        - Rey       | 0101
    * Un casillero vacio se representa mediante: 1111 1111
</pre>
----------------------------------------
### Detalle del algoritmo:
<pre>
El algoritmo consta de 4 partes:
        
    1. Deteccion del tablero: Cuando la foto se captura el tablero puede estar torcido y puesto en cualquier tipo de posicion. Por 
    lo que lo primero es identificarlo, recortarlo, centrarlo y alinearlo.

    2. Luego se reconocen cada una de las casillas individuales. Se determinan sus dimensiones, mediante 2 puntos, la esquina 
    superior izquierda y la esquina inferior izquierda.

    3. Se aplica 2 mascaras. Una de ellas detecta piezas blancas, la otra negras. Si se detecta una pieza se almacena en un array de 
    64 elementos booleanos.

    4. Se compara el array de booleanos con el de piezas y se realizan las debidas actualizaciones.  

A tener en cuenta: 

Para el correcto funcionamiento del algoritmo se debe conocer la posicion inicial de las piezas, ya que en base a estas se realiza 
la deduccion de posiciones despues.
</pre>

In [57]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [58]:
# Cargo las imagenes del tablero b
b_pos_inicio = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-b/pos-inicial.jpeg")
b_pos_inicio2 = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-b/pos-inicial-2.jpeg")
mov1 = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-b/mov-1.jpg")
mov2= cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-b/mov-2.jpg")
mov3 = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-b/mov-3.jpg")
mov4 = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-b/mov-4.jpg")
mov5 = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-b/mov-5.jpg")
mov6 = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-b/mov-6.jpg")

In [59]:
# Funciones comunes 
def showImg(img, text):
    cv2.namedWindow(text, cv2.WINDOW_KEEPRATIO)
    cv2.imshow(text, img)
    cv2.resizeWindow(text, 700, 700)
    cv2.waitKey()
    cv2.destroyAllWindows()
    
def mostrarContornos(img,min,max, bool):
    # Aplicar deteccion de bordes utilizando Canny
    edges = cv2.Canny(img, min, max, apertureSize=3)
    if bool:
        showImg(edges, 'Detector de contornos')
    return edges

# 1. Deteccion y recorte del tablero

In [60]:
# Trabajaremos con esta posicion.
showImg(b_pos_inicio, "Posicion inicial")

#### Rotamos y recortamos el tablero

In [61]:
def rotarRecortarImagen(img):
    # Convertimos la imagen a gris
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Aplicar deteccion de bordes utilizando Canny
    edges = mostrarContornos(img_gray, 100, 300, False)

    # Encontrar las lineas presentes en la imagen utilizando la transformada de Hough
    lines = cv2.HoughLinesP(edges, 1, np.pi/180, threshold=100, minLineLength=100, maxLineGap=10)

    # Calcular el angulo promedio solo de las líneas horizontales detectadas
    angle = np.mean([np.arctan2(line[0][3] - line[0][1], line[0][2] - line[0][0]) for line in lines if abs(line[0][3] - line[0][1]) < abs(line[0][2] - line[0][0])])

    # Rotar la imagen utilizando el ángulo calculado
    (h, w) = img_gray.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle * 180 / np.pi, 1.0)
    rotated = cv2.warpAffine(img, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)

    #  Convert the image to HSV color space
    hsv_image = cv2.cvtColor(rotated, cv2.COLOR_BGR2HSV)

    # Define the lower and upper bounds of the color #778078 in HSV space
    lower_color = np.array([0, 0, 0])
    upper_color = np.array([255, 90, 200])

    # Create a mask that identifies pixels that are within the range of the color #778078
    mask = cv2.inRange(hsv_image, lower_color, upper_color)

    # Display the new image
    contorno,_ = cv2.findContours(mask,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    cont = 0

    booleano = True

    for c in contorno:
        area = cv2.contourArea(c)
        if (area > 40000):
            cont = cont + 1
            x,y,w,h = cv2.boundingRect(c)
            # Crop the image using the bounding box
            if (booleano):
                cropped_image = rotated[y:(y+h), x:(x+w)].copy()
                booleano = False
            cv2.rectangle(rotated,(x,y),(x+w,y+h),(0,255,255),2)
            cv2.putText(rotated,str(cont),(x,y-5),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,255,255),2)
            area = cv2.contourArea(c)
    return cropped_image

### Tests

In [62]:
cropped_image = rotarRecortarImagen(b_pos_inicio) # no funciona
showImg(cropped_image, 'cropped_image')
cropped_image = rotarRecortarImagen(b_pos_inicio2) # no funciona
showImg(cropped_image, 'cropped_image')
cropped_image = rotarRecortarImagen(mov1) # funciona
showImg(cropped_image, 'cropped_image')
cropped_image = rotarRecortarImagen(mov2) # no funciona
showImg(cropped_image, 'cropped_image')
cropped_image = rotarRecortarImagen(mov3) # funciona
showImg(cropped_image, 'cropped_image')
cropped_image = rotarRecortarImagen(mov4) # funciona
showImg(cropped_image, 'cropped_image')
cropped_image = rotarRecortarImagen(mov5) # funciona
showImg(cropped_image, 'cropped_image')
cropped_image = rotarRecortarImagen(mov6) # funciona
showImg(cropped_image, 'cropped_image')

--------
De aca para abajo, hay codigo muy desorganizado ...

In [63]:
# pruebas de mascaras hsv para detectar el tablero
hsv = cv2.cvtColor(b_pos_inicio, cv2.COLOR_BGR2HSV)

lower_white = np.array([0, 0, 50], dtype='uint8')
upper_white = np.array([50, 50, 100], dtype='uint8')

mascara = cv2.inRange(hsv, lower_white, upper_white)

showImg(mascara, "Mascara")

In [64]:
# Convert the image to grayscale
grayscale_image = cv2.cvtColor(b_pos_inicio, cv2.COLOR_BGR2GRAY).copy()

showImg(grayscale_image, 'grayscale_image')
# Apply a Canny edge detector to the image
edges = cv2.Canny(grayscale_image, 100, 500).copy()

# Find the contours in the image
contours, hierarchy = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

# Filter the contours to find the ones that match the size and shape of a chessboard
chessboard_contours = []
for contour in contours:
    area = cv2.contourArea(contour)
    approx = cv2.approxPolyDP(contour, 0.02 * cv2.arcLength(contour, True), True)
    if len(approx) == 4 and area > 1000:
        chessboard_contours.append(contour)

# Draw the detected chessboard on the image
for contour in chessboard_contours:
    cv2.drawContours(b_pos_inicio, [contour], -1, (0, 255, 0), 2).copy()

# Display the image
showImg(b_pos_inicio, 'Image')


## Recorte y rotacion del tablero

In [65]:
# Convertimos la imagen a gris
pos_inicial_esp_gray = cv2.cvtColor(b_pos_inicio, cv2.COLOR_BGR2GRAY)

# Aplicar deteccion de bordes utilizando Canny
edges = mostrarContornos(pos_inicial_esp_gray, 200, 500)

# Encontrar las lineas presentes en la imagen utilizando la transformada de Hough
lines = cv2.HoughLinesP(edges, 1, np.pi/180, threshold=100, minLineLength=100, maxLineGap=10)

# Calcular el angulo promedio solo de las líneas horizontales detectadas
angle = np.mean([np.arctan2(line[0][3] - line[0][1], line[0][2] - line[0][0]) for line in lines if abs(line[0][3] - line[0][1]) < abs(line[0][2] - line[0][0])])

# Rotar la imagen utilizando el ángulo calculado
(h, w) = pos_inicial_esp_gray.shape[:2]
center = (w // 2, h // 2)
M = cv2.getRotationMatrix2D(center, angle * 180 / np.pi, 1.0)
rotated = cv2.warpAffine(pos_inicial_esp_gray, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)

## NOTA: Ignorar el angle2 y rotated2, solo son pruebas ===============================================================
angle2 = np.mean([np.arctan2(line[0][3] - line[0][1], line[0][2] - line[0][0]) for line in lines if abs(line[0][3] - line[0][1]) < abs(line[0][2] - line[0][0])]) + 0.009
(h, w) = pos_inicial_esp_gray.shape[:2]
center = (w // 2, h // 2)
M = cv2.getRotationMatrix2D(center, angle2 * 180 / np.pi, 1.0)
rotated2 = cv2.warpAffine(pos_inicial_esp_gray, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
## ====================================================================================================================

# Mostrar la imagen original y la imagen rotada
fig, ax = plt.subplots(1, 3, figsize=(10, 5))
ax[0].imshow(cv2.cvtColor(pos_inicial_esp_gray, cv2.COLOR_BGR2RGB))
ax[0].set_title('Original')
ax[0].axis('off')
ax[1].imshow(cv2.cvtColor(rotated, cv2.COLOR_BGR2RGB))
ax[1].set_title('Rotada y recortada')
ax[1].axis('off')
ax[2].imshow(cv2.cvtColor(rotated2, cv2.COLOR_BGR2RGB))
ax[2].set_title('Rotada - 0.001 y recortada')
ax[2].axis('off')
plt.show()



# cv2.namedWindow('custom window', cv2.WINDOW_KEEPRATIO)
# cv2.imshow('custom window', edges)
# cv2.resizeWindow('custom window', 1000, 1000)
# cv2.waitKey(4000)
# cv2.destroyAllWindows()

## ==========================================================================
#   NOTAS: 
# 1. 
# cv2.namedWindow('custom window', cv2.WINDOW_KEEPRATIO)
# cv2.imshow('custom window', tablero_recorte)
# cv2.resizeWindow('custom window', 1000, 1000)
# cv2.waitKey()
# cv2.destroyAllWindows()
 
# 2.
# Find the chessboard corners
# ret, corners = cv2.findChessboardCorners(pos_inicial_esp_gray, (9, 9))
# print(ret, corners)
# # Draw the chessboard corners on the image
# if ret: 
#   cv2.drawChessboardCorners(pos_inicial_esp, (9, 9), corners, ret)

# 3.
# amarillobajo = np.array([15,50,50],np.uint8)
# amarilloalto = np.array([25,255,255],np.uint8)
# mascara1 = cv2.inRange(imghsv,amarillobajo, amarilloalto)
# 
# verdebajo = np.array([36,50,50],np.uint8)
# verdealto = np.array([75,255,255],np.uint8)
# mascara2 = cv2.inRange(imghsv,verdebajo,verdealto)


TypeError: mostrarContornos() missing 1 required positional argument: 'bool'

In [ ]:
# Carga las imagenes en una variable
a_pos_inicio = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-a/pos-inicio.jpeg")
pos_mid = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-a/pos-mid.jpeg")
pos_mid2 = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-a/pos-mid2.jpeg")
pos_ataque = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-a/pos-ataque.jpeg")
iluminada = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-a/iluminada.jpeg")
pos_inicial_esp = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-a/pos-inicio-esp.jpeg")
pos_mid_esp = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-a/pos-mid-esp.jpeg")
pos_mid2_esp = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-a/pos-mid2-esp.jpeg")
tablero_normal = cv2.imread('/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-ideal/tablero-comun.png')

In [ ]:
## Utilizaremos rotated como la imagen base para empezar a detectar la posicion de tablero
# cv2.namedWindow('custom window', cv2.WINDOW_KEEPRATIO)
# cv2.imshow('custom window', rotated)
# cv2.resizeWindow('custom window', 1000, 1000)
# cv2.waitKey()
# cv2.destroyAllWindows()

alto, ancho = 99,89

casilleros = []

gap = 447
x = [0, 89, 199, 306, 416, 545, 654, 760]
y = [0, 99, 213, 320, 430, 539, 650, 759, ]

for fil in range(8):
    for col in range(8):
        casilleros.append(rotated[y[col]:y[col+1], x[fil]:x[fil+1]].copy())
        cv2.imshow(f"Imagen {fil},{col}", casilleros[fil * 8 + col])
        cv2.waitKey(0)
        cv2.destroyAllWindows()

In [ ]:
# procesamos la imagen 
cv2.imshow('Pos mid', pos_mid)
cv2.waitKey(5000)
cv2.destroyAllWindows()

# Convert the image to grayscale
pos_mid_gray = cv2.cvtColor(pos_mid, cv2.COLOR_BGR2GRAY)

cv2.imshow('Pos mid grayscale', pos_mid_gray)
cv2.waitKey(5000)
cv2.destroyAllWindows()

# Find the chessboard corners
ret, corners = cv2.findChessboardCorners(pos_mid_gray, (9, 9))

print(ret, corners)
cv2.imshow('tablero normal: ',tablero_normal)
cv2.waitKey(10000)
cv2.destroyAllWindows()


# Draw the chessboard corners on the image
if ret:
  cv2.drawChessboardCorners(pos_mid, (9, 9), corners, ret)

# Display the image
cv2.imshow('edges detected', pos_mid) 
cv2.waitKey(5000)
cv2.destroyAllWindows()

In [ ]:
## Imagen pos mid esp32
tablero = cv2.imread('/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-comun.jpeg')

# procesamos la imagen 
cv2.imshow('Pos mid ESP32', tablero)
cv2.waitKey(5000)
cv2.destroyAllWindows()

# Convert the image to grayscale
pos_mid_gray = cv2.cvtColor(tablero, cv2.COLOR_BGR2GRAY)

cv2.imshow('Pos mid grayscale', pos_mid_gray)
cv2.waitKey(5000)
cv2.destroyAllWindows()

# Find the chessboard corners
ret, corners = cv2.findChessboardCorners(pos_mid_gray, (7,7), cv2.CALIB_CB_ADAPTIVE_THRESH)
print(ret, corners)

# Draw the chessboard corners on the image
if ret:
  cv2.drawChessboardCorners(pos_mid_gray, (7, 7), corners, ret)

# Display the image
cv2.imshow('edges detected', pos_mid_gray) 
cv2.waitKey(5000)
cv2.destroyAllWindows()

In [ ]:
## Version 2

nline = 7
ncol = 7

## termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

## processing
gray = cv2.cvtColor(pos_mid_esp, cv2.COLOR_BGR2GRAY)

# Find the chessboard corners
ret, corners = cv2.findChessboardCorners(gray, (nline, ncol), None)
corners2 = cv2.cornerSubPix(gray, corners, (7, 7), (-1, -1), criteria)


In [ ]:
import numpy as np
import cv2 as cv
import glob
# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
images = glob.glob('*.jpg')
for fname in images:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, (7,6), None)
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners2)
        # Draw and display the corners
        cv.drawChessboardCorners(img, (7,6), corners2, ret)
        cv.imshow('img', img)
        cv.waitKey(500)
cv.destroyAllWindows()

# Posicion inicial

In [ ]:
# Muestra la imagen en una ventana
plt.imshow(mid_iluminada2)
plt.show()

In [ ]:
# Indica las coordenadas del área a recortar en píxeles
x1, y1 = 425, 95  # Esquina superior izquierda
x2, y2 = 1300, 990  # Esquina inferior derecha

# Recorta la imagen en base a las coordenadas indicadas
imagen_recortada = mid_iluminada2[y1:y2, x1:x2].copy()

# Muestra la imagen recortada en una ventana
plt.imshow(imagen_recortada)
plt.show()

gray = cv2.cvtColor(imagen_recortada, cv2.COLOR_BGR2GRAY)
cv2_imshow(gray)

dst = cv2.calcHist(gray, [0], None, [256], [0,256])

plt.hist(dst.ravel(),256,[0,256])
plt.title('Histogram for gray scale image')
plt.show()
#imghsv = cv2.cvtColor(imagen_recortada,cv2.COLOR_BGR2HSV)

#blancobajo = np.array([160,50,50],np.uint8)
#blancoalto = np.array([180,255,255],np.uint8)
#mascara1 = cv2.inRange(imghsv, blancobajo, blancoalto)

#cv2_imshow(mascara1)
#mascara1 = ~mascara1
#cv2_imshow(mascara1)

In [ ]:
x1, y1 = 3, 0  # Esquina superior izquierda
x2, y2 = 100, 105  # Esquina inferior derecha
casilla = imagen_recortada[y1:y2, x1:x2].copy()

plt.imshow(casilla)
plt.show()

imghsv = cv2.cvtColor(casilla,cv2.COLOR_BGR2HSV)

amarillobajo = np.array([15,50,50],np.uint8)
amarilloalto = np.array([25,255,255],np.uint8)
mascara1 = cv2.inRange(imghsv,amarillobajo, amarilloalto)

verdebajo = np.array([36,50,50],np.uint8)
verdealto = np.array([75,255,255],np.uint8)
mascara2 = cv2.inRange(imghsv,verdebajo,verdealto)
cv2_imshow(mascara1)



In [ ]:
x1, y1 = 148, 10  # Esquina superior izquierda
x2, y2 = 270, 138  # Esquina inferior derecha
casilla = imagen_recortada[y1:y2, x1:x2].copy()


plt.imshow(casilla)
plt.show()

imghsv = cv2.cvtColor(casilla,cv2.COLOR_BGR2HSV)

amarillobajo = np.array([15,50,50],np.uint8)
amarilloalto = np.array([25,255,255],np.uint8)
mascara1 = cv2.inRange(imghsv,amarillobajo, amarilloalto)

verdebajo = np.array([36,50,50],np.uint8)
verdealto = np.array([75,255,255],np.uint8)
mascara2 = cv2.inRange(imghsv,verdebajo,verdealto)
cv2_imshow(mascara1)



In [ ]:
x1, y1 = 410, 20  # Esquina superior izquierda
x2, y2 = 510, 128   # Esquina inferior derecha
casilla = imagen_recortada[y1:y2, x1:x2].copy()

imghsv = cv2.cvtColor(casilla,cv2.COLOR_BGR2HSV)

blancobajo = np.array([15,50,50],np.uint8)
blancoalto = np.array([25,255,255],np.uint8)
mascara1 = cv2.inRange(imghsv,blancobajo, blancoalto)
mascara1 = ~mascara1

plt.imshow(mascara1, cmap="gray")
plt.show()

contorno,_ = cv2.findContours(mascara1,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
cont = 0

for c in contorno:
    area = cv2.contourArea(c)
    if (area > 60):
        cont = cont + 1
        x,y,w,h = cv2.boundingRect(c)
        cv2.rectangle(casilla,(x,y),(x+w,y+h),(0,255,255),2)
        cv2.putText(casilla,str(cont),(x,y-5),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,255,255),2)

cv2_imshow(casilla)

if len(contorno) > 0:
    print("Se ha detectado una pieza.")
else:
    print("No se ha detectado una pieza.")

plt.imshow(mascara1)
plt.show()


In [ ]:
x1, y1 = 415, 150  # Esquina superior izquierda
x2, y2 = 515, 255  # Esquina inferior derecha
casilla1 = imagen_recortada[y1:y2, x1:x2].copy()

imghsv = cv2.cvtColor(casilla1,cv2.COLOR_BGR2HSV)

blancobajo = np.array([160,50,50],np.uint8)
blancoalto = np.array([180,255,255],np.uint8)
mascara1 = cv2.inRange(imghsv, blancobajo, blancoalto)

mascara1 = ~mascara1

contorno,_ = cv2.findContours(mascara1,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
cont = 0
for c in contorno:
    area = cv2.contourArea(c)
    if (area > 50):
        cont = cont + 1
        x,y,w,h = cv2.boundingRect(c)
        cv2.rectangle(casilla1,(x,y),(x+w,y+h),(0,255,255),2)
        cv2.putText(casilla1,str(cont),(x,y-5),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,255,255),2)

cv2_imshow(casilla1)

if len(contorno) > 0:
    print("Se ha detectado una pieza.")
else:
    print("No se ha detectado una pieza.")

plt.imshow(casilla1)
plt.show()

# Posicion de middle game


# procesamiento-tablero-v2

In [ ]:
import numpy as np 
import cv2
import sys
import os


nfila = 8
ncol = 8

img = cv2.imread('/content/pos-inicial.jpg')

# El algoritmo se detiene despues de 30 iteraciones o cuando el error es de menos de 0.001
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# Convierto la imagen a gris
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

cv2_imshow(gray)

# Devuelve ret que es un booleano que determina si se encontro un tablero y cornes es una 
# lista de puntos en la imagen que representan los vertices 
ret, corners = cv2.findChessboardCorners(gray, (nfila,ncol), None)

print(corners)


# Refina las coordenadas del tablero,
if ret == True:
  corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1,-1), criteria)


